### 1. 升级boto3, sagemaker python sdk

In [1]:
!pip install --upgrade boto3 sagemaker sagemaker_ssh_helper

In [4]:
#导入对应的库

import re
import os
import json
import uuid

import numpy as np
import pandas as pd
from time import gmtime, strftime


import boto3
import sagemaker

from sagemaker import get_execution_role,session
from sagemaker import Model, image_uris, serializers, deserializers

role = get_execution_role()
sage_session = session.Session()
bucket = sage_session.default_bucket()
aws_region = boto3.Session().region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
print(f'account id:{account_id}')

print(f'sagemaker sdk version: {sagemaker.__version__}\nrole:  {role}  \nbucket:  {bucket}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
account id:434444145045
sagemaker sdk version: 2.232.2
role:  arn:aws:iam::434444145045:role/test2-ExecutionRole-R1gUnbXC9ZOp  
bucket:  sagemaker-us-west-2-434444145045


### 2. 编译docker image (comfyui-inference)

In [5]:
## You should change below region code to the region you used, here sample is use us-west-2
# !aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

In [93]:
# !bash build_and_push.sh docker/Dockerfile_deploy

In [94]:
# !touch dummy
# !tar czvf model.tar.gz dummy
# assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'stablediffusion')
# model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'stablediffusion')
# !aws s3 cp model.tar.gz $assets_dir
# !rm -f dummy model.tar.gz

### 下载flux模型并上传到S3

In [80]:
from pathlib import Path
local_model_path = Path("./flux-model")
local_model_path.mkdir(exist_ok=True)

!wget -O ./flux-model/flux1-dev-bnb-nf4-v2.safetensors https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors 

--2024-10-10 03:33:32--  https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.59, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/22/5b/225be068416fc1776856b384b9f3482592328892f6dd54ca5fe279b4c52f3817/fef37763b8f2c8cc3463139bbb6d91aa517c605b654d4e5c540de52813f30306?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux1-dev-bnb-nf4-v2.safetensors%3B+filename%3D%22flux1-dev-bnb-nf4-v2.safetensors%22%3B&Expires=1728790412&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODc5MDQxMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzIyLzViLzIyNWJlMDY4NDE2ZmMxNzc2ODU2YjM4NGI5ZjM0ODI1OTIzMjg4OTJmNmRkNTRjYTVmZTI3OWI0YzUyZjM4MTcvZmVmMzc3NjNiOGYyYzhjYzM0NjMxMzliYmI2ZDkxYWE1MTdjNj

In [10]:
import sagemaker
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name
s3_model_prefix = "flux-models"  # folder where model checkpoint will go



In [82]:
!aws s3 cp --recursive ./flux-model s3://{bucket}/{s3_model_prefix}

upload: flux-model/flux1-dev-bnb-nf4-v2.safetensors to s3://sagemaker-us-west-2-434444145045/flux-models/flux1-dev-bnb-nf4-v2.safetensors


In [78]:
!aws s3 ls s3://{bucket}/{s3_model_prefix}/

2024-10-09 15:41:43   18319825 flux1-dev-bnb-nf4-v2.safetensors


In [11]:
models = {"s3_bucket": f"{bucket}", "MODEL_PATH": f"s3://{bucket}/{s3_model_prefix}"}
models

{'s3_bucket': 'sagemaker-us-west-2-434444145045',
 'MODEL_PATH': 's3://sagemaker-us-west-2-434444145045/flux-models'}

### 部署模型

In [12]:
import jinja2
from jinja2 import Template
import os
import time

node_urls = ['https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4']

def build_image():
    print("build_image====")
    global models, node_urls
    deploy_output = ""
    # 读取 Dockerfile 模板
    with open('dockerfile.template', 'r') as file:
        template_content = file.read()
    # 创建 Jinja2 模板对象
    template = Template(template_content)
    # 定义要克隆的 Git 仓库 URL 列表
    git_urls = node_urls
    # 生成 git clone 命令列表
    git_clone_commands = []
    for url in git_urls:
        repo_name = os.path.splitext(os.path.basename(url))[0]
        command = f"RUN git clone {url} /opt/program/custom_nodes/{repo_name}"
        git_clone_commands.append(command)
    # 渲染模板
    rendered_content = template.render(git_clone_commands=git_clone_commands)
    # 将渲染后的内容写入新的 Dockerfile
    with open('Dockerfile_deploy', 'w') as file:
        file.write(rendered_content)

    #gr.Info("New Dockerfile has been generated.")
    print("New Dockerfile has been generated.")

    ## step1: build docker image
    deploy_output=f"开始build镜像（初次build会下载base image，有一定的时间，请耐心等待)"
    #gr.Info(deploy_output)
    print(deploy_output)

    # AWS ECR login
    os.system("aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com")
    # Build and push
    os.system("./build_and_push.sh Dockerfile_deploy")

# 部署模型
def deploy_model(instance_type, region):
    # Create dummy file and tar
    with open('dummy', 'w') as f:
        pass
    os.system("tar czvf model.tar.gz dummy")
    # Set S3 paths
    assets_dir = f's3://{bucket}/stablediffusion/assets/'
    model_data = f's3://{bucket}/stablediffusion/assets/model.tar.gz'
    # Upload to S3
    os.system("aws s3 cp model.tar.gz "+assets_dir)
    # Clean up
    os.remove('dummy')
    os.remove('model.tar.gz')

    ## step2: create sagemaker model config
    env = models
    print(env)
    container=f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/comfyui-inference:latest"

    model = Model(image_uri=container,
              model_data=model_data,
              role=role,
              env=env,
              #dependencies=[SSHModelWrapper.dependency_dir()]
              )

    deploy_output = f"开始部署模型\n实例类型: {instance_type}\n区域: {region}\n"
    #gr.Info(deploy_output)
    print(deploy_output)

    ## step3: start deployment
    endpoint_name = f"comfyui-endpoint-{int(time.time())}"
    try:
        deploy_output = f"正在创建 SageMaker endpoint: {endpoint_name}\n"
        print(deploy_output)
        model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name,
             container_startup_health_check_timeout=2600
            )
        deploy_output += f"{endpoint_name}部署成功!\n"
        #gr.Info(deploy_output)
        print(deploy_output)
        print("here2====")
        return endpoint_name
    except Exception as e:
        print("here3====")
        print(e)
        deploy_output += f"部署过程中出现错误: {str(e)}\n"
        print(deploy_output)
        return endpoint_name

### build 镜像

In [2]:
build_image()

build_image====
New Dockerfile has been generated.
开始build镜像（初次build会下载base image，有一定的时间，请耐心等待)


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

fatal: destination path 'ComfyUI' already exists and is not an empty directory.
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile_deploy
#1 DONE 0.0s


Login Succeeded



#1 [internal] load build definition from Dockerfile_deploy
#1 transferring dockerfile: 1.82kB done
#1 DONE 0.0s

#2 [internal] load metadata for nvcr.io/nvidia/pytorch:23.05-py3
#2 DONE 0.5s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [ 1/28] FROM nvcr.io/nvidia/pytorch:23.05-py3@sha256:d5aa1e516e68afab9cd3ecaaeac3dd2178618bd26cd7ad96762ed53e32e9e0bd
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 29.56kB 0.0s done
#5 DONE 0.0s

#6 [23/28] RUN mkdir -p /opt/program/models/xlabs/loras
#6 CACHED

#7 [ 5/28] RUN pip install sagemaker-ssh-helper
#7 CACHED

#8 [10/28] RUN pip install -U xformers
#8 CACHED

#9 [18/28] RUN wget https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors
#9 CACHED

#10 [16/28] RUN pip install bitsandbytes
#10 CACHED

#11 [21/28] RUN git clone https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4 /opt/program/custom_nodes/ComfyUI_bitsandbytes_NF4
#11 CACHED

#12 [ 3/28

The push refers to repository [434444145045.dkr.ecr.us-west-2.amazonaws.com/comfyui-inference]
5f70bf18a086: Preparing
8561ab695c97: Preparing
4c102b3b386d: Preparing
cec0b36a3c89: Preparing
a144c9d746a1: Preparing
a52ce2251f46: Preparing
5d9a364842ad: Preparing
68fecde52660: Preparing
e517da2e68cd: Preparing
9aeab4c64788: Preparing
f5190a4a52c7: Preparing
4157985c3eb2: Preparing
4a7da64c63df: Preparing
2109a0dc0b92: Preparing
5588a9e44b78: Preparing
a660e6973195: Preparing
0518724b63f1: Preparing
0c4dfd1acb82: Preparing
5b9b245fa845: Preparing
481b8ba2a2cc: Preparing
b59126ae116b: Preparing
f15c037aedda: Preparing
acad10c3c71e: Preparing
2d706a9cd37e: Preparing
52ef53958d0b: Preparing
af5bd1acd6a0: Preparing
25e08d43b658: Preparing
34e30321ba09: Preparing
f684be5caa97: Preparing
9b11fc966566: Preparing
a5bc09d60ea2: Preparing
7411a27d7ac3: Preparing
60ee97d95a35: Preparing
c8a46bb5d982: Preparing
f3f637c3407a: Preparing
4e81a870531b: Preparing
21cfa7f07daf: Preparing
975d84760b95: Pre

In [13]:
instance_type = "ml.g4dn.2xlarge"
endpoint_name = deploy_model(instance_type,aws_region)

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-434444145045/stablediffusion/assets/model.tar.gz
{'s3_bucket': 'sagemaker-us-west-2-434444145045', 'MODEL_PATH': 's3://sagemaker-us-west-2-434444145045/flux-models'}
开始部署模型
实例类型: ml.g4dn.2xlarge
区域: us-west-2

正在创建 SageMaker endpoint: comfyui-endpoint-1728539826

--------------------!正在创建 SageMaker endpoint: comfyui-endpoint-1728539826
comfyui-endpoint-1728539826部署成功!

here2====
